# 환경 사용

지금까지 Azure Machine Learning 작업 영역에서 여러 실험을 실행했으며, 일부 실험에서는 실험 코드가 실행되는 환경에서 필요한 특정 Python 패키지를 지정해야 했습니다. 이 랩에서는 환경을 좀 더 자세히 살펴봅니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험용 데이터 준비

이 랩에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀을 실행하여 이 데이터 세트를 만듭니다. 이전 랩에서 해당 데이터 세트를 만든 경우 코드를 실행하면 새 버전이 작성됩니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                       target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                       overwrite=True, # 이름이 같은 기존 파일 바꾸기
                       show_progress=True)

# 데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 테이블 형식 데이터 세트 등록
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset ready.')

## 학습 스크립트 만들기

아래의 두 셀을 실행하여 다음 항목을 만듭니다.
1. 새 실험용 폴더
2. **scikit-learn**을 사용하여 모델을 학습시키고 **matplotlib**를 사용하여 ROC 곡선을 그리는 학습 스크립트 파일

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 정규화 하이퍼 매개 변수 설정(스크립트에 인수로 전달됨)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 환경 정의

Azure Machine Learning에서 Python 스크립트를 실험으로 실행하면 스크립트용 실행 컨텍스트를 정의하는 Conda 환경이 작성됩니다. Azure Machine Learning은 여러 공통 패키지가 포함된 기본 환경을 제공합니다. 이러한 패키지로는 실험 실행 사용에 필요한 라이브러리가 포함된 **azureml-defaults** 패키지, 그리고 **pandas**/**numpy** 등의 널리 사용되는 패키지 등이 있습니다.

**conda** 또는 **pip**를 사용하여 자체 환경을 정의하고 패키지를 추가할 수도 있습니다. 그러면 실험에서 필요한 모든 라이브러리에 액세스할 수 있습니다. 

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# 실험용 Python 환경 만들기
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Azure ML의 종속성 관리 허용
diabetes_env.docker.enabled = True # Docker 컨테이너 사용

# 패키지 종속성 집합 만들기(필요에 따라 conda 또는 pip)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                                          pip_packages=['azureml-sdk','pyarrow'])

# 환경에 종속성 추가
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

이제 추정기나 RunConfig에 환경을 할당하여 실험에 환경을 사용할 수 있습니다.

다음 코드는 작성된 환경을 일반 추정기에 할당하고 실험을 제출합니다. 실험이 실행되면 위젯과 **azureml_logs/60_control_log.txt** 출력 로그에서 실행 세부 정보를 관찰합니다. 그러면 conda 환경이 작성되고 있음을 확인할 수 있습니다.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 스크립트 매개 변수 설정
script_params = {
    '--regularization': 0.1
}

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 추정기 만들기
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # 데이터 세트를 입력으로 전달
              script_params=script_params,
              compute_target = 'local',
              environment_definition = diabetes_env,
              entry_script='diabetes_training.py')

# 실험 만들기
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()
run.wait_for_completion()

실험에서는 필요한 모든 패키지가 포함된 환경을 정상적으로 사용했습니다. 아래 코드를 실행하거나 Azure Machine Learning Studio를 통해 실험 실행의 출력과 메트릭을 확인할 수 있습니다. 예를 들어 **scikit-learn**을 사용하여 학습시킨 모델, **matplotlib**를 사용하여 생성된 ROC 차트 이미지 등을 확인할 수 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 환경 등록

필요한 패키지가 포함된 환경을 정의한 후에는 작업 영역에 해당 환경을 등록할 수 있습니다.

In [ ]:
# 환경 등록
diabetes_env.register(workspace=ws)

환경은 처음 만들 때 할당한 이름(여기서는 *diabetes-experiment-env*)으로 등록됩니다.

환경을 등록하면 요구 사항이 같은 모든 스크립트에 해당 환경을 재사용할 수 있습니다. 여기서는 환경 재사용의 예를 확인하기 위해 다른 알고리즘을 사용하여 당뇨병 모델을 학습시키는 스크립트와 폴더를 만들어 보겠습니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

이제 등록된 환경을 검색한 다음 대체 학습 스크립트를 실행하는 새 실험용 추정기를 구성하는 데 사용할 수 있습니다. 단, 의사 결정 트리 분류기에는 하이퍼 매개 변수 값이 필요하지 않으므로 이번에는 스크립트 매개 변수가 없습니다.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails

registered_env = Environment.get(ws, 'diabetes-experiment-env')

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 추정기 만들기
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # 데이터 세트를 입력으로 전달
              compute_target = 'local',
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# 실험 만들기
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()
run.wait_for_completion()

이번에는 실험이 더 빠르게 실행됩니다. 이전 실행에서 일치하는 환경이 캐시되었으므로 로컬 컴퓨팅에서 환경을 다시 만들 필요가 없기 때문입니다. 하지만 컴퓨팅 대상이 다르더라도 같은 환경이 작성되어 사용됩니다. 그러므로 실험 스크립트 실행 컨텍스트의 일관성이 유지됩니다.

이번에는 실험의 메트릭과 출력을 살펴보겠습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이 모델은 로지스틱 회귀 모델보다 성능이 다소 우수해 보이므로 등록해 보겠습니다.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator + Environment (Decision Tree)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 등록된 환경 확인

직접 만든 환경을 등록할 수도 있고, 일반 실험 유형에는 미리 작성된 "큐레이트" 환경을 활용할 수도 있습니다. 다음 코드는 등록된 모든 환경의 목록을 표시합니다.

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

모든 큐레이트 환경의 이름은 ***AzureML-***로 시작됩니다. 직접 만든 환경에는 이 접두사를 사용할 수 없습니다.

큐레이트 환경을 더 자세히 살펴보고 각 환경에 포함된 패키지를 확인해 보겠습니다.

In [ ]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

> **추가 정보**: Azure Machine Learning의 환경에 대한 자세한 내용은 [Azure Machine Learning 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)를 참조하세요.